# Lesson 5: Event-driven generation

### Import all needed packages

In [1]:
import boto3, os

from helpers.Lambda_Helper import Lambda_Helper
from helpers.S3_Helper import S3_Helper

lambda_helper = Lambda_Helper()
s3_helper = S3_Helper()

bucket_name_text = os.environ['LEARNERS3BUCKETNAMETEXT']
bucket_name_audio = os.environ['LEARNERS3BUCKETNAMEAUDIO']

### Deploy your lambda function

In [2]:
%%writefile lambda_function.py

#############################################################
#
# This Lambda function is written to a file by the notebook 
# It does not run in the notebook!
#
#############################################################

import json
import boto3
import uuid
import os

s3_client = boto3.client('s3')
transcribe_client = boto3.client('transcribe', region_name='us-west-2')

def lambda_handler(event, context):
    # Extract the bucket name and key from the incoming event
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    # One of a few different checks to ensure we don't end up in a recursive loop.
    if key != "dialog.mp3": 
        print("This demo only works with dialog.mp3.")
        return

    try:
        
        job_name = 'transcription-job-' + str(uuid.uuid4()) # Needs to be a unique name

        response = transcribe_client.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': f's3://{bucket}/{key}'},
            MediaFormat='mp3',
            LanguageCode='en-US',
            OutputBucketName= os.environ['S3BUCKETNAMETEXT'],  # specify the output bucket
            OutputKey=f'{job_name}-transcript.json',
            Settings={
                'ShowSpeakerLabels': True,
                'MaxSpeakerLabels': 2
            }
        )
        
    except Exception as e:
        print(f"Error occurred: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error occurred: {e}")
        }

    return {
        'statusCode': 200,
        'body': json.dumps(f"Submitted transcription job for {key} from bucket {bucket}.")
    }



Writing lambda_function.py


In [3]:
lambda_helper.lambda_environ_variables = {'S3BUCKETNAMETEXT' : bucket_name_text}
lambda_helper.deploy_function(["lambda_function.py"], function_name="LambdaFunctionTranscribe")

Zipping function...
Looking for existing function...
Function LambdaFunctionTranscribe does not exist. Creating...
Function LambdaFunctionTranscribe created: arn:aws:lambda:us-west-2:621952221782:function:LambdaFunctionTranscribe
Done.


In [4]:
lambda_helper.filter_rules_suffix = "mp3"
lambda_helper.add_lambda_trigger(bucket_name_audio, function_name="LambdaFunctionTranscribe")

Permission added with Statement: {
    "Sid": "s3-trigger-permission",
    "Effect": "Allow",
    "Principal": {
        "Service": "s3.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": "arn:aws:lambda:us-west-2:621952221782:function:LambdaFunctionTranscribe",
    "Condition": {
        "ArnLike": {
            "AWS:SourceArn": "arn:aws:s3:::c99355a2566048l6711959t1w0702-learneraudios3bucket-52ljdrqxw7ac"
        }
    }
}
Trigger added for c99355a2566048l6711959t1w0702-learneraudios3bucket-52ljdrqxw7ac -> LambdaFunctionTranscribe


In [5]:
s3_helper.upload_file(bucket_name_audio, 'dialog.mp3')

Object 'dialog.mp3' uploaded to bucket 'c99355a2566048l6711959t1w0702-learneraudios3bucket-52ljdrqxw7ac'


In [6]:
s3_helper.list_objects(bucket_name_audio)

Object: dialog.mp3, Created on: 2025-08-24 08:17:17+00:00


#### Restart kernel if needed.
- If you run the code fairly quickly from start to finish, it's possible that the following code cell `s3_helper.list_objects(bucket_name_text)` will give a "Not Found" error.  
- If waiting a few seconds (10 seconds) and re-running this cell does not resolve the error, then you can restart the kernel of the jupyter notebook.
- Go to menu->Kernel->Restart Kernel.
- Then run the code cells from the start of the notebook, waiting 2 seconds or so for each code cell to finish executing.

In [9]:
s3_helper.list_objects(bucket_name_text)

Object: .write_access_check_file.temp, Created on: 2025-08-24 08:17:20+00:00
Object: results.txt, Created on: 2025-08-24 08:17:48+00:00
Object: transcription-job-9e18b821-1853-456e-a587-e193a864cbe6-transcript.json, Created on: 2025-08-24 08:17:41+00:00


#### Re-run "download" code cell as needed
- It may take a few seconds for the results to be generated.
- If you see a `Not Found` error, please wait a few seconds and then try running the `s3_helper.download_object` again.

In [10]:
s3_helper.download_object(bucket_name_text, 'results.txt')

Object 'results.txt' from bucket 'c99355a2566048l6711959t1w07026-learnertexts3bucket-ovxztreqdyic' to './results.txt'


In [11]:
from helpers.Display_Helper import Display_Helper

In [12]:
display_helper = Display_Helper()

In [13]:
display_helper.text_file('results.txt')

results.txt:


Extra resources:

* [Generative AI code](https://community.aws/code/generative-ai)

* [Generative AI](https://community.aws/generative-ai)
